In [1]:
include("/home/kwat/github/Kraft.jl/src/Kraft.jl")

Main.Kraft

In [2]:
read_gmt

UndefVarError: UndefVarError: read_gmt not defined

In [2]:
const genes = readlines("/home/kwat/github/kraft/notebook/genes.txt")[2:end];

const scores = fill(
    1 / length(genes),
    length(genes),
);

const gene_set_dict = read_gmt("/home/kwat/garden/data/gene_set/msigdb_v6.2/h.all.v6.2.symbols.gmt")

const gene_set_genes = sort(gene_set_dict[sort(collect(keys(gene_set_dict)))[1]]);

UndefVarError: UndefVarError: read_gmt not defined

In [ ]:
using BenchmarkTools

In [ ]:
const gene_rank = Dict(zip(
    genes,
    1:length(genes),
))

const hits = make_hits(
    genes,
    gene_set_genes,
)

In [ ]:
function compute_gene_set_enrichment(
    genes::Array{String, 1},
    scores::Array{Float64, 1},
    gene_set_genes::Array{String, 1},
    hits::Array{Int64, 1},
)
    
    # 1ns
    n = length(genes)
    
    # 2us
    cumulative_sum = Array{Float64, 1}(
        undef,
        n,
    )
    
    # 13us
    hit_scores_sum = sum_hit_scores(
        scores,
        hits,
    )
    
    # 6us
    d_down = -1 / (n - sum_hits(hits))
    
    # 0.01ns
    value = 0.0
    
    # 0.01ns
    auc = 0.0
    
    # 0.01ns
    min_ = 0.0
    
    # 0.01ns
    max_ = 0.0
    
    @inbounds @fastmath @simd for i in 1:n
        
        if hits[i] == 1
            
            value += abs(scores[i]) / hit_scores_sum
            
        else
            
            value += d_down
            
        end
        
        # 50us
        cumulative_sum[i] = value
        
        auc += value
        
        if value < min_
            
            min_ = value
            
        elseif max_ < value
            
            max_ = value
            
        end
            
    end
    
    cumulative_sum, auc, min_, max_
    
end


@benchmark compute_gene_set_enrichment(
    genes,
    scores,
    gene_set_genes,
    hits,
)

In [ ]:
using PyCall

In [ ]:
pd = pyimport("pandas")

kraft = pyimport("kraft")

In [ ]:
gene_score = pd.Series(
    scores,
    index=genes,
)

In [ ]:
println(kraft.run_single_sample_gsea(
    gene_score,
    gene_set_genes,
    hit=hits,
    plot=false,
))

@benchmark kraft.run_single_sample_gsea(
    gene_score,
    gene_set_genes,
    hit=hits,
    plot=false,
)